# Milestone Writeup

In [ ]:
Data Acquisition and Description
We got our data from Oikolab. They get their data from various sources, such as NOAA, GEFS, CHIRPS, and ERA5. The data came in a csv file, with a variety of variables to choose from, all of which we explored in our autoregression file, in which we decided which variables to focus on in our correlation matrix. 
Milestones:
Linear regression models
	Previously done
        -Preprocessing data
        -Encoded categorical data
        -OLS Summary for each city
        -Mean snowfall for February by year by city graphs
	Need to do
        -Add a regression line to plots
        -Change x-axis ticks
        -Add comments to this code
Auto regression models
	Previously done
        -Preprocessing data
        -Encoded categorical data
        -Fit an auto-regression model with the SLC all Param data
        -Get a predicted snowfall for Feb 10 2034 (which day is arbitrary)
	Need to do
        -Get an accuracy for regression model/other statistical analysis of the autoregression
        -Find another model to make a comparison
        -Add bullet for prediction for feb 2034 to other plots 
        -Possibly run other city datasets through the auto regression model.
        -Clean code


	
Data
    Previously done
        -Reformat data/other preprocessing. 
        -Create a correlation matrix to identify which variable correlates most with Snowfall in SLC
        -Find historic mean, upper limit, lower limit  for Feb 1994-2024 in SLC
    Need to do
        -Show statistical significance or insignificance of data and plots.
        -Make sure all datasets cover the same years.


#Simple Linear Regression

In [ ]:
import pandas as pd

ogden = "Ogden_Data.csv"
provo = "Provo_Data.csv"
slc = "SLC_All_Param.csv"
ogden_df = pd.read_csv(ogden)
provo_df = pd.read_csv(provo)
slc_df = pd.read_csv(slc)

In [ ]:
ogden_df['datetime (UTC)'] = pd.to_datetime(ogden_df['datetime (UTC)'])
ogden_df[['latitude', 'longitude']] = ogden_df['coordinates (lat,lon)'].str.extract(r'\(([^,]+), ([^)]+)\)').astype(float)

provo_df['datetime (UTC)'] = pd.to_datetime(provo_df['datetime (UTC)'])
provo_df[['latitude', 'longitude']] = provo_df['coordinates (lat,lon)'].str.extract(r'\(([^,]+), ([^)]+)\)').astype(float)

slc_df['datetime (UTC)'] = pd.to_datetime(slc_df['datetime (UTC)'])
slc_df[['latitude', 'longitude']] = slc_df['coordinates (lat,lon)'].str.extract(r'\(([^,]+), ([^)]+)\)').astype(float)

ogden_df_encoded = pd.get_dummies(ogden_df, columns=['model (name)'])
provo_df_encoded = pd.get_dummies(provo_df, columns=['model (name)'])
slc_df_encoded = pd.get_dummies(slc_df, columns=['model (name)'])

ogden_df_encoded['hour'] = ogden_df_encoded['datetime (UTC)'].dt.hour
ogden_df_encoded['day'] = ogden_df_encoded['datetime (UTC)'].dt.day
ogden_df_encoded['month'] = ogden_df_encoded['datetime (UTC)'].dt.month
ogden_df_encoded['year'] = ogden_df_encoded['datetime (UTC)'].dt.year

provo_df_encoded['hour'] = provo_df_encoded['datetime (UTC)'].dt.hour
provo_df_encoded['day'] = provo_df_encoded['datetime (UTC)'].dt.day
provo_df_encoded['month'] = provo_df_encoded['datetime (UTC)'].dt.month
provo_df_encoded['year'] = provo_df_encoded['datetime (UTC)'].dt.year

slc_df_encoded['hour'] = slc_df_encoded['datetime (UTC)'].dt.hour
slc_df_encoded['day'] = slc_df_encoded['datetime (UTC)'].dt.day
slc_df_encoded['month'] = slc_df_encoded['datetime (UTC)'].dt.month
slc_df_encoded['year'] = slc_df_encoded['datetime (UTC)'].dt.year

ogden_df_encoded.drop(['datetime (UTC)', 'coordinates (lat,lon)'], axis=1, inplace=True)
provo_df_encoded.drop(['datetime (UTC)', 'coordinates (lat,lon)'], axis=1, inplace=True)
slc_df_encoded.drop(['datetime (UTC)', 'coordinates (lat,lon)'], axis=1, inplace=True)

In [ ]:
features = ['temperature (degC)', 'relative_humidity (0-1)', 'total_precipitation (mm of water equivalent)', 'total_cloud_cover (0-1)']
target_variable = 'snowfall (mm of water equivalent)'

ogden_data = ogden_df_encoded[['month', 'year', target_variable] + features].copy()
provo_data = provo_df_encoded[['month', 'year', target_variable] + features].copy()
slc_data = slc_df_encoded[['month', 'hour', 'year', target_variable] + features].copy()

In [ ]:
ogden_filtered_data = ogden_data[ogden_data['month'] == 2]
provo_filtered_data = provo_data[provo_data['month'] == 2]
slc_filtered_data = slc_data[(slc_data['hour'] == 10) & (slc_data['month'] == 2)]

slc_yearly_mean = slc_filtered_data.groupby('year').mean()

slc_yearly_mean = slc_yearly_mean.reset_index()

In [ ]:
import statsmodels.api as sm

ogden_x = ogden_filtered_data['year']
ogden_y = ogden_filtered_data['snowfall (mm of water equivalent)']
ogden_result = sm.OLS(ogden_y, ogden_x).fit()

provo_x = provo_filtered_data['year']
provo_y = provo_filtered_data['snowfall (mm of water equivalent)']
provo_result = sm.OLS(provo_y, provo_x).fit()

slc_x = slc_yearly_mean['year']
slc_y = slc_yearly_mean['snowfall (mm of water equivalent)']
slc_result = sm.OLS(slc_y, slc_x).fit()

print("Ogden OLS")
print(ogden_result.summary())
print("Provo OLS")
print(provo_result.summary())
print("Salt Lake City OLS")
print(slc_result.summary())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(ogden_x, ogden_y, marker='o')

plt.title('Mean snowfall for February by Year in Ogden')
plt.xlabel('Year')
plt.ylabel('Mean snowfall (mm of water equivalent)')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(provo_x, provo_y, marker='o')

plt.title('Mean snowfall for February by Year in Provo')
plt.xlabel('Year')
plt.ylabel('Mean snowfall (mm of water equivalent)')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(slc_x, slc_y, marker='o')

plt.title('Mean snowfall for February by Year in Salt Lake City')
plt.xlabel('Year')
plt.ylabel('Mean snowfall (mm of water equivalent)')
plt.grid(True)
plt.show()

# Auto Regression

In [ ]:
import pandas as pd

file_path = 'SLC_all_param.csv'
df = pd.read_csv(file_path)

In [ ]:
#Preprocessing for all features

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

# Parse datetime and split coordinates
df['datetime (UTC)'] = pd.to_datetime(df['datetime (UTC)'])
df[['latitude', 'longitude']] = df['coordinates (lat,lon)'].str.extract(r'\(([^,]+), ([^)]+)\)').astype(float)

# Drop columns with 50% or more null values
df = df.dropna(thresh=len(df) * 0.7, axis=1)

# Encode categorical variables
df_encoded = pd.get_dummies(df, columns=['model (name)'])

# break down the UTC t from datetime
df_encoded['hour'] = df_encoded['datetime (UTC)'].dt.hour
df_encoded['day'] = df_encoded['datetime (UTC)'].dt.day
df_encoded['month'] = df_encoded['datetime (UTC)'].dt.month

# Drop the original datetime and coordinates columns
df_encoded.drop(['datetime (UTC)', 'coordinates (lat,lon)'], axis=1, inplace=True)

# Check if df_encoded is empty
#  print(df_encoded.head())

# Scaling numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_encoded.select_dtypes(include=['float64', 'int64']))  # Scale only numerical features
df_scaled = pd.DataFrame(scaled_features, columns=df_encoded.select_dtypes(include=['float64', 'int64']).columns)
df_encoded[df_scaled.columns] = df_scaled

# Calculate correlation with target variable
correlation_with_target = df_encoded.corr()['snowfall (mm of water equivalent)'].abs()

# Select top 5 most influential features (excluding the target variable)
top_5_features = correlation_with_target.sort_values(ascending=False).index[1:6]

print("Top 5 features and their correlation with the target variable:")
for feature in top_5_features:
    correlation_value = correlation_with_target[feature]
    print(f"{feature}: {correlation_value:.2f}")

# Create correlation matrix using only the top 5 features and the target variable
correlation_matrix = df_encoded[top_5_features].join(df_encoded['snowfall (mm of water equivalent)']).corr()

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"fontsize":10, "weight": "bold"})
plt.title("Correlation Matrix of Top 5 Most Influential Features")
plt.tight_layout()
plt.show()

In [ ]:
top_10_features = correlation_with_target.sort_values(ascending=False).index[1:11]

# Print top 10 features and their correlation values
print("Top 10 features and their correlation with the target variable:")
for feature in top_10_features:
    correlation_value = correlation_with_target[feature]
    print(f"{feature}: {correlation_value:.2f}")

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Parse datetime and split coordinates
df['datetime (UTC)'] = pd.to_datetime(df['datetime (UTC)'])

# Select features and target variable
features = ['temperature (degC)', 'relative_humidity (0-1)', 'total_precipitation (mm of water equivalent)', 'total_cloud_cover (0-1)']
target_variable = 'snowfall (mm of water equivalent)'

data = df[['datetime (UTC)', target_variable] + features].copy()

data['hour'] = data['datetime (UTC)'].dt.hour
data['day'] = data['datetime (UTC)'].dt.day
data['month'] = data['datetime (UTC)'].dt.month
data['year'] = data['datetime (UTC)'].dt.year

data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
data.set_index('date', inplace=True)

# Drop the original datetime and coordinates columns
data.drop(['datetime (UTC)'], axis=1, inplace=True)

print(len(data))

In [ ]:
filtered_data = data[(data['hour'] == 10) & (data['month'] == 2)]

# Create a DataFrame from the filtered data
df = pd.DataFrame(filtered_data.copy())
df2 = df.copy()

snowfall_mean = df['snowfall (mm of water equivalent)'].mean()

print("Mean snowfall (mm of water equivalent) in febs (1994-2024):", snowfall_mean)

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error

# Assuming filtered_data contains the DataFrame you provided earlier

# Define the features and target variable
features = ['temperature (degC)', 'relative_humidity (0-1)', 'total_precipitation (mm of water equivalent)', 'total_cloud_cover (0-1)', 'date']
target_variable = 'snowfall (mm of water equivalent)'

# Recreate the index with a specific frequency (assuming daily frequency)
df.index = pd.date_range(start=df.index.min(), periods=len(df), freq='D')

# Fit autoregression model
model = AutoReg(df[target_variable], lags=1)  # Assuming lag order is 1
model_fit = model.fit()

# Make predictions
predictions = model_fit.predict(start='2034-02-10', end='2034-02-10', dynamic=False)

print(predictions)

print(df.head())


In [ ]:
# Define the date for which you want to make the prediction
prediction_date = '2034-02-28'
prediction_date = '2034-02-28'
# Predict snowfall for the specified date
prediction = model_fit.predict(start=prediction_date, end=prediction_date)

print("Predicted snowfall on February 10, 2034:", prediction.values[0])


In [ ]:
# Calculate summary statistics from historical data
historical_mean = df['snowfall (mm of water equivalent)'].mean()
historical_std = df['snowfall (mm of water equivalent)'].std()

# Compare with predicted snowfall
predicted_snowfall = prediction.values[0]  # Your predicted snowfall value
significance_level = 0.95  # For example, use 95% confidence interval

# Calculate the upper and lower bounds of significance
upper_bound = historical_mean + significance_level * historical_std
lower_bound = historical_mean - significance_level * historical_std

print("Historic Mean:")
print(historical_mean)

print("Upper:")
print(upper_bound)
print("Lower:")
print(lower_bound)
print("Predicted:")
print(predicted_snowfall)


In [ ]:
# Filter the data for February (month = 2)
february_data = df2[df2['month'] == 2]

# Group by year, calculate the mean
yearly_mean = february_data.groupby('year').mean()

# Reset index to make 'year' as a column
yearly_mean = yearly_mean.reset_index()

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(yearly_mean['year'], yearly_mean['snowfall (mm of water equivalent)'], marker='o')

plt.title('Mean snowfall for February by Year')
plt.xlabel('Year')
plt.ylabel('Mean snowfall (mm of water equivalent)')
plt.grid(True)
plt.show()

In [ ]:
# Find historical maximum and minimum snowfall values
historical_max = df2['snowfall (mm of water equivalent)'].max()
historical_min = df2['snowfall (mm of water equivalent)'].min()

# Find dates corresponding to the maximum and minimum snowfall values
date_max = df2[df2['snowfall (mm of water equivalent)'] == historical_max].index[0]
date_min = df2[df2['snowfall (mm of water equivalent)'] == historical_min].index[0]

# Print the maximum and minimum values along with their corresponding dates
print("Historical Maximum Snowfall:", historical_max, "mm")
print("Date of Maximum Snowfall:", date_max)
print("Historical Minimum Snowfall:", historical_min, "mm")
print("Date of Minimum Snowfall:", date_min)